**Chat Model**


In [4]:
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryMemory(llm=llm)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


def get_history():
    return memory.load_memory_variables({})


add_message("Hi I'm Maverick, I'm in Topgun", "Wow, that is so cool!")

In [5]:
add_message("Do you have anything to ask?", "yes, how's Iceman doing?")

get_history()

{'history': "The human introduces themselves as Maverick from Topgun. The AI responds with enthusiasm, saying it's cool and asks how Iceman is doing."}